# TP4bis : Deep Learning appliqué aux images

## Les images
 - Les couches utilisées par les méthodes Deep Learning sur les images : Vocabulaire + concepts
 - Application : Implémenter le réseau LeNet-5

![](https://miro.medium.com/max/700/1*ueA-rooOaiIo3s2rVVz3Ww.png)

# Les layers pour les images

## Les convolutions
-> Leur but : Extraire des features locales des images

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d

=> https://github.com/vdumoulin/conv_arithmetic


## Les poolings
-> leur but : Diminuer la taille de l'output

https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d


## Les DropOuts
-> Leur but : Diminuer le sur-apprentissage (overfitting)

https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html#torch.nn.Dropout


## Les fonctions d'activation
-> Leur but : Introduire des non linéarités

https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU

https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html#torch.nn.Tanh

https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html#torch.nn.Sigmoid

# Implémenter le réseau LeNet-5

![](https://miro.medium.com/max/700/1*ueA-rooOaiIo3s2rVVz3Ww.png)

Les clefs pour décoder ce schéma :
- Cx — convolution layer,
- Sx — subsampling (pooling) layer,
- Fx — fully-connected layer, (La Linear de la dernière fois)
- x — index of the layer.

-> le petit détail en + il y a une fonction d'activation après chaque convolution et chaque layer linear.

### Implémentez le réseau LeNet5 en pytorch

In [5]:
import torch
import torch.nn as nn

class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x

## Data Loaders

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

from IPython.display import display
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

BATCH_SIZE = 64
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define transforms
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor()])

# download and create datasets
train_dataset = datasets.MNIST(root='',
                               train=True,
                               transform=transform)

valid_dataset = datasets.MNIST(root='',
                               train=False,
                               transform=transform)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False)

RuntimeError: Dataset not found. You can use download=True to download it

## Helpers to train and evaluate the model

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### A vous de jouer

Assemblez les éléments pour entrainer le modèle, évaluer ses performances

In [ ]:
model = LeNet5()

# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 10000
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epoch)

test(model, device, valid_loader)

## Predict !

In [ ]:
img, classe = random.choice(train_loader.dataset)
print(classe)
display(transforms.ToPILImage()(img))
predict(model, img)